In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
vidObj = cv.VideoCapture("./videos/ghoda.mp4")
fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter('./output/ghoda.avi',fourcc, 30.0, (640,480))

In [ ]:
imgArray = []
maskArray = []

In [ ]:
%%time

success = 1
i=1

success, img1 = vidObj.read()
while success:
    img = cv.resize(img1, dsize=(640,480), interpolation=cv.INTER_LINEAR)
    mask = np.zeros(img.shape[:2],np.uint8)
    
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)
    rect = (100,30,300,400)
    
    cv.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv.GC_INIT_WITH_RECT);
    front_mask = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    fore = img*front_mask[:,:,np.newaxis]
    out.write(fore)
    print(i)
    i +=1
    
    imgArray.append(img)
    maskArray.append(mask)
    
    success, img1 = vidObj.read()

vidObj.release()
out.release()
print("Ended")

In [ ]:
maskArray = np.asarray(maskArray)

In [ ]:
imgArray = np.asarray(imgArray)

In [ ]:
imgArray.shape , maskArray.shape

In [ ]:
filt = np.ones([3,3,3])

In [ ]:
# 1 in foreground
mask = np.where((maskArray==2)|(maskArray==0),0,1).astype('uint8')

In [ ]:
from scipy.ndimage import correlate

In [ ]:
c = correlate(mask,filt,mode = 'nearest')

In [ ]:
# if number of pixel in neighbourhood >= 15
maskNew = np.where((c >= 15),1,0).astype('uint8')

In [ ]:
newout = cv.VideoWriter('./output/smoothghoda.avi',fourcc, 30.0, (640,480))

In [ ]:
for i in range(imgArray.shape[0]):
    img = imgArray[i,:,:,:]
    front_mask = maskNew[i,:,:]
    fore = img*front_mask[:,:,np.newaxis]
    newout.write(fore)
newout.release()